# 26/04/2023

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# import mongo client to connect  
#from pymongo import MongoClient 
#import pprint  

import sqlite3

from datetime import datetime

In [2]:
#Opciones de navegacion

options =  webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')

driver_path = Service("C:\\Users\\Tatu\\Downloads\\chromedriver_win32\\chromedriver.exe")

driver = webdriver.Chrome(service = driver_path, options = options)


In [3]:

driver.get('https://es.investing.com/equities/ypf-sociedad')

tit=driver.title
print(tit)


Acciones YPF | Cotización BA:YPFD hoy - Investing.com


In [4]:
# Insertar fecha y hora actual
now = datetime.now()
print(now)

2023-05-05 23:11:14.474165


In [5]:
ultimo_cierre = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[2]/div/div[1]/div/div[5]/div[1]/dl[1]/div[1]/dd/span/span[1]')
ultimo_cierre = ultimo_cierre.text
print("Último cierre " + ultimo_cierre)

rango_min = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[2]/div/div[1]/div/div[5]/div[1]/dl[1]/div[3]/dd/span[1]/span[1]')
rango_min = rango_min.text
print("Mínimo " + rango_min)

rango_max = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[2]/div/div[1]/div/div[5]/div[1]/dl[1]/div[3]/dd/span[3]/span[1]')
rango_max = rango_max.text
print("Máximo " + rango_max)

ingresos_T = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[2]/div/div[1]/div/div[5]/div[1]/dl[2]/div[2]/dd/span/span[1]')
ingresos_T = ingresos_T.text
print("Ingresos " + ingresos_T + "T")

apertura = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[2]/div/div[1]/div/div[5]/div[1]/dl[1]/div[2]/dd/span/span[1]')
apertura = apertura.text
print("Apertura " + apertura)


volumen = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[2]/div/div[1]/div/div[5]/div[1]/dl[1]/div[5]/dd/span/span[1]')
volumen = volumen.text
print("Volumen " + volumen)

volumen_promedio = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[2]/div/div[1]/div/div[5]/div[1]/dl[1]/div[6]/dd/span/span[1]')
volumen_promedio = volumen_promedio.text
print("Volumen promedio " + volumen_promedio)

Último cierre 4.768,75
Mínimo 4.841
Máximo 5.229,9
Ingresos 2,53T
Apertura 4.841
Volumen 322.677
Volumen promedio 304.817


In [6]:
driver.close()

In [7]:
#Conexión con la base de datos
conn = sqlite3.connect('acciones_YPFD.db')

In [8]:
#Creación de la tabla "resumen"
c = conn.cursor()

c.execute('''CREATE TABLE resumen
             (date_time DATETIME, ultimo_precio REAL, minimo REAL, maximo REAL, ingresos REAL, apertura REAL, volumen REAL, volumen_promedio REAL)''')

In [9]:
# Insertar datos en la tabla
c.execute("INSERT INTO resumen (date_time, ultimo_precio, minimo, maximo, ingresos, apertura, volumen, volumen_promedio) VALUES (?, ?, ?, ?, ?, ?, ?, ?)", 
          (now, ultimo_cierre, rango_min, rango_max, ingresos_T, apertura, volumen, volumen_promedio))

# Guardar los cambios en la base de datos y cerrar la conexión
conn.commit()


In [10]:
#Vemos los datos guardados
# Seleccionar los datos de la tabla
c.execute("SELECT * FROM resumen")

# Obtener los resultados de la consulta
resultados = c.fetchall()

# Imprimir los resultados en la consola
for fila in resultados:
    print(fila)

('2023-05-05 23:11:14.474165', '4.768,75', 4.841, '5.229,9', '2,53', 4.841, 322.677, 304.817)


In [11]:
# Cerrar la conexión a la base de datos
conn.close()